# NLP Group Project
### Alberto Puentes, Parker Voit, Tyler Applegate
#### Florence Cohort, 2021_08_03

In [1]:
# imports
from requests import get
from bs4 import BeautifulSoup
import os
import re
import numpy as np
import pandas as pd
import acquire
import markdown 
import prepare

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

page = requests.get('https://github.com/topics/awesome')

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')